# Geemap

[Geemap](https://geemap.org/) is a Python package to visually analyze geospatial data with Google Earth Engine (GEE).

This snippet shows how to render a `GeoDataFrame` returned by a UDF in geemap. Specifically, the [UDF](https://github.com/fusedio/udfs/tree/main/public/Overture_Maps_Example) loads Buildings from the [Overture](https://beta.source.coop/repositories/fused/overture/description/) dataset, converts them to a GEE [FeatureCollection](https://developers.google.com/earth-engine/apidocs/ee-featurecollection), and renders them on the map as vectors.

![geemap](https://fused-magic.s3.us-west-2.amazonaws.com/docs_assets/geemap.png)

In [ ]:
!pip install fused geemap earthengine-api

## 1. Authenticate into Google Earth Engine

Running this command should prompt you to open a window to get then provide the GEE token.

In [ ]:
import geemap
import ee
import fused

ee.Initialize()

## 2. Run Overture UDF to get Buildings GeoDataFrame

In [ ]:
# Load UDF
udf = fused.load("https://github.com/fusedio/udfs/tree/main/public/Overture_Maps_Example")

# Run UDF
gdf_buildings = fused.run(udf=udf, x=10484, y=25324, z=16, engine="local")
gdf_buildings = gdf_buildings.set_crs("WGS84")
gdf_buildings.head()

## Render on `geemap` map

In [ ]:

# Create a map centered at the given location
Map = geemap.Map(center=(37.8, -122.4), zoom=14)

# Render buildings on map
fc = geemap.geopandas_to_ee(gdf_buildings)
Map.addLayer(fc, {'color': 'red'}, "Default Overture Buildings")
Map

## Bonus: Perform a spatial query for custom geometry

First render the map and draw a single polygon for the region of interest.

In [19]:
# Create a map centered at the given location
Map = geemap.Map(center=(37.8, -122.4), zoom=14)
Map

Map(center=[37.8, -122.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [ ]:
# Generate a GeoDataFrame of the drawn polygon
roi = ee.FeatureCollection(Map.draw_features)
gdf_bbox = geemap.ee_to_gdf(roi)

# Run UDF
gdf_buildings = udf(bbox=gdf_bbox).run_local()

# Render buildings on map
fc = geemap.geopandas_to_ee(gdf_buildings)
Map.addLayer(fc, {}, "Custom Overture Buildings")